# Couchbase Analytics

## Setup

In [13]:
# needed for any cluster connection
from couchbase.cluster import Cluster, ClusterOptions
from couchbase_core.cluster import PasswordAuthenticator

# needed to support SQL++ (N1QL) query
from couchbase.cluster import QueryOptions

In [14]:
from couchbase.exceptions import CouchbaseException
from couchbase.cluster import AnalyticsOptions

In [15]:
import pandas as pd
import json
import os

In [16]:
user = os.environ['CB_USER']
password = os.environ['CB_PASSWORD']
host_url = os.environ['CB_HOST_URL']

In [17]:
# get a reference to our cluster
cluster = Cluster(host_url, ClusterOptions(
  PasswordAuthenticator(user, password)))

## Analytics

In [25]:
query = "SELECT car.ModelType, COUNT(*) AS num_types FROM cars car GROUP BY car.ModelType ORDER BY COUNT(*) DESC;"

In [26]:
json_string = "{"
counter = 0
try:
    result = cluster.analytics_query(query)

    for row in result.rows():
        if counter != 0:
            json_string += ","
        counter += 1
        json_string += "'" + str(counter) + "':" + str(row)

except CouchbaseException as ex:
    import traceback
    traceback.print_exc()
json_string += "}"

In [27]:
json_string

"{'1':{'num_types': 18, 'ModelType': 'Large SUV'},'2':{'num_types': 9, 'ModelType': 'Compact car'},'3':{'num_types': 9, 'ModelType': 'Convertible'},'4':{'num_types': 9, 'ModelType': 'Coupe'},'5':{'num_types': 9, 'ModelType': 'Hybrid'},'6':{'num_types': 9, 'ModelType': 'Medium SUV'},'7':{'num_types': 9, 'ModelType': 'Mid-size'},'8':{'num_types': 9, 'ModelType': 'Sports car'},'9':{'num_types': 9, 'ModelType': 'Station wagon'},'10':{'num_types': 1}}"

In [28]:
result.rows()

[{'num_types': 18, 'ModelType': 'Large SUV'},
 {'num_types': 9, 'ModelType': 'Compact car'},
 {'num_types': 9, 'ModelType': 'Convertible'},
 {'num_types': 9, 'ModelType': 'Coupe'},
 {'num_types': 9, 'ModelType': 'Hybrid'},
 {'num_types': 9, 'ModelType': 'Medium SUV'},
 {'num_types': 9, 'ModelType': 'Mid-size'},
 {'num_types': 9, 'ModelType': 'Sports car'},
 {'num_types': 9, 'ModelType': 'Station wagon'},
 {'num_types': 1}]

In [29]:
s = pd.read_json(json_string.replace("'", '"'))

In [30]:
s.T

,num_types,ModelType
1,18,Large SUV
2,9,Compact car
3,9,Convertible
4,9,Coupe
5,9,Hybrid
6,9,Medium SUV
7,9,Mid-size
8,9,Sports car
9,9,Station wagon
10,1,NaN
